<a href="https://colab.research.google.com/github/stfromnjust/nlp-task/blob/main/%E2%80%9CTransformer_Torch%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.manythings.org/anki/cmn-eng.zip
!unzip -d ./cmn-eng cmn-eng.zip

--2021-04-12 14:28:40--  http://www.manythings.org/anki/cmn-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3031::6815:37de, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1062383 (1.0M) [application/zip]
Saving to: ‘cmn-eng.zip’

cmn-eng.zip         100%[===================>]   1.01M  1.10MB/s    in 0.9s    

2021-04-12 14:28:41 (1.10 MB/s) - ‘cmn-eng.zip’ saved [1062383/1062383]

Archive:  cmn-eng.zip
  inflating: ./cmn-eng/cmn.txt       
  inflating: ./cmn-eng/_about.txt    


In [ ]:
# 每一行数据如下
# 'Hi.\t嗨。\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #891077 (Martha)'
with open('./cmn-eng/cmn.txt', 'r', encoding='utf-8') as f:
    data = f.read()
data = data.strip()
data = data.split('\n')
print('样本数:\n', len(data))
print('\n样本示例:')
data[0]

样本数:
 24360

样本示例:


'Hi.\t嗨。\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #891077 (Martha)'

In [ ]:
# 分割英文数据和中文数据
en_data = [line.split('\t')[0] for line in data]
ch_data = [line.split('\t')[1] for line in data]
print('英文数据:\n', en_data[:10])
print('\n中文数据:\n', ch_data[:10])

英文数据:
 ['Hi.', 'Hi.', 'Run.', 'Wait!', 'Wait!', 'Begin.', 'Hello!', 'I won!', 'Oh no!', 'Cheers!']

中文数据:
 ['嗨。', '你好。', '你用跑的。', '等等！', '等一下！', '开始！', '你好。', '我赢了。', '不会吧。', '乾杯!']


In [ ]:
# 按字符级切割，并添加<eos>
en_token_list = [[char for char in line] for line in en_data]
ch_token_list = [[char for char in line] for line in ch_data]
print('英文数据:\n', en_token_list[:10])
print('\n中文数据:\n', ch_token_list[:10])

英文数据:
 [['H', 'i', '.'], ['H', 'i', '.'], ['R', 'u', 'n', '.'], ['W', 'a', 'i', 't', '!'], ['W', 'a', 'i', 't', '!'], ['B', 'e', 'g', 'i', 'n', '.'], ['H', 'e', 'l', 'l', 'o', '!'], ['I', ' ', 'w', 'o', 'n', '!'], ['O', 'h', ' ', 'n', 'o', '!'], ['C', 'h', 'e', 'e', 'r', 's', '!']]

中文数据:
 [['嗨', '。'], ['你', '好', '。'], ['你', '用', '跑', '的', '。'], ['等', '等', '！'], ['等', '一', '下', '！'], ['开', '始', '！'], ['你', '好', '。'], ['我', '赢', '了', '。'], ['不', '会', '吧', '。'], ['乾', '杯', '!']]


In [ ]:
def list2sentence(list_):
    ret = ''
    for i in range(len(list_) - 1):
        ret += list_[i]
        ret += '_'
    ret += list_[len(list_) - 1]
    return ret


def padding(sentence, padding_num):
    for i in range(padding_num):
        sentence += '_<P>'
    return sentence
  
ch_len_list = [len(sen) for sen in ch_token_list]
en_len_list =  [len(sen) for sen in en_token_list]
ch_max_len = max(ch_len_list)
en_max_len = max(en_len_list)
print('maxch', ch_max_len)
print('maxen', en_max_len)
senten = []
for i in range(len(en_token_list)):
  ret = []
  padding_en = list2sentence(en_token_list[i])
  padding_en = padding(padding_en, en_max_len - len(en_token_list[i]))
  # if i < 30:
  #   print(padding_en)
  #   print(len(padding_en.split('_')))
  ret.append(padding_en)
  padding_ch = list2sentence(ch_token_list[i])
  padding_ch = padding(padding_ch, ch_max_len - len(ch_token_list[i]))
  ret.append('<S>_' + padding_ch)
  ret.append(padding_ch + '_<E>')
  senten.append(ret)
# print(senten[:10])  

en_vocab = set(''.join(en_data))

en2id = {(''+char):i+1 for i, char in enumerate(en_vocab)}
en2id['<P>'] = 0
ch_vocab = set(''.join(ch_data))

ch2id = {(''+char):i+1 for i, char in enumerate(ch_vocab)}
ch2id['<P>'] = 0
ch2id['<S>'] = len(ch2id)
ch2id['<E>'] = len(ch2id)
print(en2id)
print(ch2id)




maxch 44
maxen 163
{'"': 1, 'r': 2, 'u': 3, '?': 4, '5': 5, 'U': 6, 'h': 7, 'Z': 8, '$': 9, 'v': 10, 'é': 11, 'n': 12, '0': 13, 'M': 14, '7': 15, '8': 16, 'E': 17, 'A': 18, 'd': 19, 'J': 20, 'D': 21, 'p': 22, 'g': 23, 'I': 24, 'L': 25, 'N': 26, 'Q': 27, 'W': 28, 'K': 29, 'T': 30, 'G': 31, '-': 32, 'a': 33, '%': 34, '9': 35, 'H': 36, 'i': 37, 'B': 38, '.': 39, 'F': 40, 'R': 41, 'e': 42, 's': 43, '’': 44, '3': 45, 'O': 46, 'f': 47, 'b': 48, '2': 49, 'c': 50, 'l': 51, '6': 52, 'q': 53, 'x': 54, '!': 55, "'": 56, ':': 57, 'S': 58, 'o': 59, 'V': 60, '4': 61, ' ': 62, 'Y': 63, 'm': 64, 'P': 65, 'j': 66, 'k': 67, ',': 68, 'C': 69, 'w': 70, 't': 71, '1': 72, 'z': 73, 'y': 74, '+': 75, '<P>': 0}
{'妄': 1, '國': 2, '饶': 3, '？': 4, '精': 5, '雄': 6, '岁': 7, '脆': 8, '網': 9, '同': 10, '军': 11, '汤': 12, '嘿': 13, '佣': 14, '鍋': 15, '宵': 16, '怒': 17, '谢': 18, '针': 19, '稠': 20, '瞟': 21, '丈': 22, '簿': 23, '詢': 24, '蜜': 25, '概': 26, '补': 27, '命': 28, '鳴': 29, '罰': 30, '逃': 31, '蜍': 32, '示': 33, '御': 34, '太': 3

In [ ]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, Derek Miller @dmmiller612, modify by wmathor
  Reference : https://github.com/jadore801120/attention-is-all-you-need-pytorch
              https://github.com/JayParks/transformer
'''
import math
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps
sentences = [
        # enc_input           dec_input         dec_output
        ['ich mochte ein bier P', 'S i want a beer .', 'i want a beer . E'],
        ['ich mochte ein cola P', 'S i want a coke .', 'i want a coke . E']
]
sentences = senten
print(len(sentences))
# Padding Should be Zero
src_vocab = {'P' : 0, 'ich' : 1, 'mochte' : 2, 'ein' : 3, 'bier' : 4, 'cola' : 5}
src_vocab = en2id
src_vocab_size = len(src_vocab)

tgt_vocab = {'P' : 0, 'i' : 1, 'want' : 2, 'a' : 3, 'beer' : 4, 'coke' : 5, 'S' : 6, 'E' : 7, '.' : 8}
tgt_vocab = ch2id
idx2word = {tgt_vocab[key]:key for key in tgt_vocab}
print(idx2word)
tgt_vocab_size = len(tgt_vocab)

src_len = 170 # enc_input max sequence length
tgt_len = 50 # dec_input(=dec_output) max sequence length

# Transformer Parameters
d_model = 512  # Embedding Size
d_ff = 2048 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

24360
{1: '妄', 2: '國', 3: '饶', 4: '？', 5: '精', 6: '雄', 7: '岁', 8: '脆', 9: '網', 10: '同', 11: '军', 12: '汤', 13: '嘿', 14: '佣', 15: '鍋', 16: '宵', 17: '怒', 18: '谢', 19: '针', 20: '稠', 21: '瞟', 22: '丈', 23: '簿', 24: '詢', 25: '蜜', 26: '概', 27: '补', 28: '命', 29: '鳴', 30: '罰', 31: '逃', 32: '蜍', 33: '示', 34: '御', 35: '太', 36: '賺', 37: '怯', 38: '奶', 39: '顿', 40: '陌', 41: '报', 42: '椒', 43: '荒', 44: '守', 45: '記', 46: '副', 47: '宠', 48: '币', 49: '解', 50: '景', 51: '紙', 52: '澤', 53: '籠', 54: '贈', 55: '席', 56: '犬', 57: '邻', 58: '穷', 59: '确', 60: '額', 61: '愛', 62: '味', 63: '穌', 64: '由', 65: '尾', 66: '晨', 67: '口', 68: '籲', 69: '忘', 70: '咆', 71: '菲', 72: '哭', 73: '蘭', 74: '滴', 75: '安', 76: '带', 77: '畔', 78: '酒', 79: '退', 80: '尽', 81: '馆', 82: '崩', 83: '揭', 84: '择', 85: '辞', 86: '癢', 87: '尼', 88: '擎', 89: '繩', 90: '扛', 91: '冲', 92: '温', 93: '濺', 94: '咽', 95: '盡', 96: '初', 97: '翁', 98: '裡', 99: '節', 100: '墩', 101: '转', 102: '积', 103: '股', 104: '蘇', 105: '㥃', 106: '狮', 107: '結', 108: '点', 109: '倚', 110: '丽', 1

In [ ]:
def make_data(sentences):
    enc_inputs, dec_inputs, dec_outputs = [], [], []
    for i in range(len(sentences)):
      enc_input = [[src_vocab[n] for n in sentences[i][0].split('_')]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
      dec_input = [[tgt_vocab[n] for n in sentences[i][1].split('_')]] # [[6, 1, 2, 3, 4, 8], [6, 1, 2, 3, 5, 8]]
      dec_output = [[tgt_vocab[n] for n in sentences[i][2].split('_')]] # [[1, 2, 3, 4, 8, 7], [1, 2, 3, 5, 8, 7]]

      enc_inputs.extend(enc_input)
      dec_inputs.extend(dec_input)
      dec_outputs.extend(dec_output)

    return torch.LongTensor(enc_inputs), torch.LongTensor(dec_inputs), torch.LongTensor(dec_outputs)

enc_inputs, dec_inputs, dec_outputs = make_data(sentences)

class MyDataSet(Data.Dataset):
  def __init__(self, enc_inputs, dec_inputs, dec_outputs):
    super(MyDataSet, self).__init__()
    self.enc_inputs = enc_inputs
    self.dec_inputs = dec_inputs
    self.dec_outputs = dec_outputs
  
  def __len__(self):
    return self.enc_inputs.shape[0]
  
  def __getitem__(self, idx):
    return self.enc_inputs[idx], self.dec_inputs[idx], self.dec_outputs[idx]

loader = Data.DataLoader(MyDataSet(enc_inputs, dec_inputs, dec_outputs), 32, True)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

def get_attn_subsequence_mask(seq):
    '''
    seq: [batch_size, tgt_len]
    '''
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1) # Upper triangular matrix
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask # [batch_size, tgt_len, tgt_len]

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).cuda()(output + residual), attn

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).cuda()(output + residual) # [batch_size, seq_len, d_model]

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        dec_enc_attn_mask: [batch_size, tgt_len, src_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        # dec_outputs: [batch_size, tgt_len, d_model], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn, dec_enc_attn

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
        dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_outputs.transpose(0, 1)).transpose(0, 1).cuda() # [batch_size, tgt_len, d_model]
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0).cuda() # [batch_size, tgt_len, tgt_len]

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs) # [batc_size, tgt_len, src_len]

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder().cuda()
        self.decoder = Decoder().cuda()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False).cuda()
    def forward(self, enc_inputs, dec_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        dec_inputs: [batch_size, tgt_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns

model = Transformer().cuda()
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.99)
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, '\t\t', param.shape)

encoder.src_emb.weight 		 torch.Size([76, 512])
encoder.layers.0.enc_self_attn.W_Q.weight 		 torch.Size([512, 512])
encoder.layers.0.enc_self_attn.W_K.weight 		 torch.Size([512, 512])
encoder.layers.0.enc_self_attn.W_V.weight 		 torch.Size([512, 512])
encoder.layers.0.enc_self_attn.fc.weight 		 torch.Size([512, 512])
encoder.layers.0.pos_ffn.fc.0.weight 		 torch.Size([2048, 512])
encoder.layers.0.pos_ffn.fc.2.weight 		 torch.Size([512, 2048])
encoder.layers.1.enc_self_attn.W_Q.weight 		 torch.Size([512, 512])
encoder.layers.1.enc_self_attn.W_K.weight 		 torch.Size([512, 512])
encoder.layers.1.enc_self_attn.W_V.weight 		 torch.Size([512, 512])
encoder.layers.1.enc_self_attn.fc.weight 		 torch.Size([512, 512])
encoder.layers.1.pos_ffn.fc.0.weight 		 torch.Size([2048, 512])
encoder.layers.1.pos_ffn.fc.2.weight 		 torch.Size([512, 2048])
encoder.layers.2.enc_self_attn.W_Q.weight 		 torch.Size([512, 512])
encoder.layers.2.enc_self_attn.W_K.weight 		 torch.Size([512, 512])
encoder.layers.2.e

In [ ]:
for epoch in range(30):
    if epoch > 0:
      print('Epoch:', '%04d' % (epoch), 'loss =', '{:.6f}'.format(total_loss))
    total_loss = 0
    for enc_inputs, dec_inputs, dec_outputs in loader:
      '''
      enc_inputs: [batch_size, src_len]
      dec_inputs: [batch_size, tgt_len]
      dec_outputs: [batch_size, tgt_len]
      '''
      enc_inputs, dec_inputs, dec_outputs = enc_inputs.cuda(), dec_inputs.cuda(), dec_outputs.cuda()
      # outputs: [batch_size * tgt_len, tgt_vocab_size]
      outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
      loss = criterion(outputs, dec_outputs.view(-1))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      total_loss += loss

In [ ]:
def greedy_decoder(model, enc_input, start_symbol):
    """
    For simplicity, a Greedy Decoder is Beam search when K=1. This is necessary for inference as we don't know the
    target sequence input. Therefore we try to generate the target input word by word, then feed it into the transformer.
    Starting Reference: http://nlp.seas.harvard.edu/2018/04/03/attention.html#greedy-decoding
    :param model: Transformer Model
    :param enc_input: The encoder input
    :param start_symbol: The start symbol. In this example it is 'S' which corresponds to index 4
    :return: The target input
    """
    enc_outputs, enc_self_attns = model.encoder(enc_input)
    dec_input = torch.zeros(1, tgt_len).type_as(enc_input.data)
    next_symbol = start_symbol
    for i in range(0, tgt_len):
        dec_input[0][i] = next_symbol
        dec_outputs, _, _ = model.decoder(dec_input, enc_input, enc_outputs)
        projected = model.projection(dec_outputs)
        prob = projected.squeeze(0).max(dim=-1, keepdim=False)[1]
        next_word = prob.data[i]
        next_symbol = next_word.item()
    return dec_input

In [ ]:
# Test
# enc_inputs, _, _ = next(iter(loader))
enc_inputs, dec_inputs, dec_outputs = make_data(sentences)
test_list = [22329, 22322, 6518, 24235, 16669, 16036, 17132, 13479, 15243, 19545]
for test_index in test_list:
  greedy_dec_input = greedy_decoder(model, enc_inputs[test_index].view(1, -1).cuda(), start_symbol=tgt_vocab["<S>"])
  predict, _, _, _ = model(enc_inputs[test_index].view(1, -1).cuda(), greedy_dec_input)
  predict = predict.data.max(1, keepdim=True)[1]
  print('[原文]')
  print(en_data[test_index])
  print('[原文]')
  print(ch_data[test_index])
  print('[机器翻译]')
  print([idx2word[n.item()] for n in predict.squeeze()])